In [ ]:
!pip install gpytorch

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch import nn
import gpytorch
from gpytorch import kernels, means, models, mlls, settings, constraints
from gpytorch import distributions as distr
import json
import scipy
import scipy.stats
import tqdm
from itertools import combinations

DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
with open('CVPR_2022_NAS_Track2_train.json', 'r') as f:
    train_data = json.load(f)
print(train_data['arch1'])
print('train_num:',len(train_data.keys()))
def convert_X(arch_str):
    temp_arch = []
    for elm in arch_str:
        if elm == 'l':
            temp_arch.append(0)
        elif elm == 'j':
            temp_arch.append(1)
        elif elm == 'k':
            temp_arch.append(2)
        else: 
            temp_arch.append(int(elm))
    return(temp_arch)
train_list = [[],[],[],[],[],[],[],[]]
arch_list_train = []
name_list = ['cplfw_rank', 'market1501_rank', 'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank', 'veriwild_rank', 'sop_rank']
for key in train_data.keys():
    for idx, name in enumerate(name_list):
        train_list[idx].append(train_data[key][name])#8*500
    arch_list_train.append(convert_X(train_data[key]['arch']))#500
print(len(train_list))

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x

In [ ]:
tmp=[0]
for i in range(1,37):
    if(i%3==0):
        continue
    tmp.append(i)

train_num = 400

In [ ]:
# Use the simplest form of GP model, exact inference
class GPModel(models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)

        self.ln_e=nn.Embedding(3, 2, padding_idx=None)
        self.depth_e=nn.Embedding(13, 2, padding_idx=None)
        self.head_e=nn.Embedding(4, 2, padding_idx=0)
        self.mlp_e=nn.Embedding(4, 2, padding_idx=0)
        
        base_kernel = kernels.MaternKernel(nu=1.5)
        
        self.mean_module = means.MultitaskMean(
            means.ConstantMean(), num_tasks=len(Y_all_k[0])
        )
        self.covar_module = kernels.MultitaskKernel(
            kernels.ArcKernel(base_kernel), num_tasks=len(Y_all_k[0]), rank=1
        )
        
        self.encoder_layer1 = nn.TransformerEncoderLayer(d_model=4, nhead=2,dim_feedforward=4,dropout=0.5,activation="relu",batch_first=True,layer_norm_eps=1e-5,norm_first=True)
        self.encoder_layer2 = nn.TransformerEncoderLayer(d_model=4, nhead=2,dim_feedforward=4,dropout=0.5,activation="relu",batch_first=True,layer_norm_eps=1e-5,norm_first=True)
        
        self.linear = nn.Linear(300,8)
        
    def forward(self, x):
        t1=self.depth_e(torch.tensor([12]).long().to(DEVICE))
        t2=self.ln_e(x[:,0].unsqueeze(1))
        
        t1=torch.stack([t1 for _ in range(len(t2))])
        
        e=torch.cat((t1,t2),axis=-1)

        for i in range(1,25):
            t1=self.depth_e(torch.tensor(int(i/2)).long().to(DEVICE)).reshape((1,-1))
            t1=torch.stack([t1 for _ in range(len(t2))])
            if((i%2)==0):
                t2=self.mlp_e(x[:,i].unsqueeze(1))
                tt=torch.cat((t1,t2),axis=-1)
                e=torch.cat((e,tt),axis=1)
            else:
                t2=self.head_e(x[:,i].unsqueeze(1))
                tt=torch.cat((t1,t2),axis=-1)
                e=torch.cat((e,tt),axis=1)
        x=e
        
        xl=[x]
        
        x=self.encoder_layer1(x)
        xl.append(x)
        x=self.encoder_layer2(x)
        xl.append(x)
        
        x=torch.cat(xl,axis=-1)
        
        x=x.squeeze()
        
        x=torch.flatten(x,1,-1)
        
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return distr.MultitaskMultivariateNormal(mean_x, covar_x)

In [ ]:
training_iterations = 500

res={"cp_l":[],"score_l":[]}

for cpnum in range(1,9):
    for cp in combinations([0,1,2,3,4,5,6,7], cpnum):
        task_list=list(cp)
        print(task_list)
        
        X_all_k, Y_all_k  = np.array(arch_list_train)[:,tmp], ZscoreNormalization(np.array(train_list).T)
        X_all_k, Y_all_k = torch.tensor(X_all_k).long().to(DEVICE),torch.tensor(Y_all_k).float()[:,task_list].to(DEVICE)
        X_train_k, Y_train_k, X_test_k, Y_test_k = X_all_k[0:train_num:1], Y_all_k[0:train_num:1], X_all_k[train_num::1], Y_all_k[train_num::1]
        X_test_k = X_test_k.clone()
        Y_test_k = Y_test_k
        train_x = X_train_k[0:400,:].clone()
        train_y = Y_train_k[0:400:,:].clone()
        
        likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=len(Y_all_k[0])).to(DEVICE)
        model = GPModel(train_x, train_y, likelihood).to(DEVICE)

        optimizer = torch.optim.Adam(model.parameters(),0.1)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
        
        model.train()
        likelihood.train()

        for i in range(training_iterations):
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, train_y)
            loss.backward()
            optimizer.step()
            
        model.eval()
        likelihood.eval()
        
        score=[0,0,0,0,0,0,0,0]
        
        with torch.no_grad():
            observed_pred = likelihood(model(X_test_k))
            
        for j in range(len(Y_all_k[0])):
            kenda, p_value = scipy.stats.stats.kendalltau(observed_pred.mean.cpu().numpy()[:,j],Y_test_k.cpu().numpy()[:,j])
            score[task_list[j]]=kenda
                
        res['cp_l'].append(task_list)
        res['score_l'].append(score)
        
with open("search_res.json", 'w') as ft:
    json.dump(res, ft)

In [ ]:
scores=np.array(res['score_l'])

first=5
for i in range(8):
    print("task",i,":")
    tmp=np.argsort(scores[:,i])
    for j in range(1,first+1):
        print(j)
        print("score:",scores[tmp[-j]][i])
        print("cp:",res['cp_l'][tmp[-j]])
        print("\n\n")